# Import the libs and base ontologies

In [1]:
from owlready2 import *
# from emmopy import get_emmo
from neo4j import GraphDatabase, basic_auth
from owlready2 import get_ontology
# from emmopy import get_emmo
from neo4j import GraphDatabase, basic_auth
from sentence_transformers import SentenceTransformer, util
import torch
import re
onto_path.append("")
onto_process=get_ontology("C://Abhishek Thesis//ontology//manufacturing.owl").load()

# onto_matter=get_ontology("C://Abhishek Thesis//ontology//matter.owl").load()
onto_matter=get_ontology("https://raw.githubusercontent.com/IEK-13/MatGraphAI/master/Ontology/matter.owl").load()

onto_quantity=get_ontology("C://Abhishek Thesis//ontology//quantities.owl").load()


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [ ]:
# results = onto_quantity.search_one(label="properties.Quantity")
# print(results)

# Creating node-class 'EMMO_IS_A' relations between exisiting classes.

## map the 3 ontologies on neo4j

In [2]:



# Define Neo4j connection details
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"

# Create Neo4j driver and session
driver = GraphDatabase.driver(uri, auth=(user, password))
session = driver.session()

# Define function to create nodes
def create_node(tx, label, name,iri):
    tx.run("MERGE (n:" + label + " {name: $name, uri:$uri})", name=name, uri=iri)
    

# Define function to create EMMO_IS_A relationships
def create_emmo_is_a_rel(tx,class_, subclass, superclass):
    tx.run("MATCH (c1:" + class_ + " {name: $subclass}), (c2:" + class_ + " {name: $superclass}) "
           "MERGE (c1)-[:EMMO_IS_A]->(c2)",class_=class_ ,subclass=subclass, superclass=superclass)

# Create nodes for each ontology
# with session.begin_transaction() as tx:
#     create_node(tx, "EEMOMatter", "Matter")
#     create_node(tx, "EMMOProcess", "Process")
#     create_node(tx, "EMMOQuantity", "Quantity")

# Create nodes for each class in Matter ontology
with session.begin_transaction() as tx:
    for clazz in onto_matter.classes():
        create_node(tx, "EMMOMatter", clazz.name,clazz.iri)

# Create nodes for each class in Process ontology
with session.begin_transaction() as tx:
    for clazz in onto_process.classes():
        create_node(tx, "EMMOProcess", clazz.name,clazz.iri)

# Create nodes for each class in Quantity ontology
with session.begin_transaction() as tx:
    for clazz in onto_quantity.classes():
        create_node(tx, "EMMOQuantity", clazz.name,clazz.iri)

# Create EMMO_IS_A relationships for each class
with session.begin_transaction() as tx:
    for clazz in onto_matter.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,'EMMOMatter', clazz.name, parent.name)
    for clazz in onto_process.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,'EMMOProcess', clazz.name, parent.name)
    for clazz in onto_quantity.classes():
        for parent in clazz.is_a:
            create_emmo_is_a_rel(tx,'EMMOQuantity', clazz.name, parent.name)

# Close Neo4j session and driver
session.close()
driver.close()


# Neo4j codes

In [ ]:
# Match(n) optional match(n)-[r]-() delete n,r

#to search for a node
# MATCH (n:Person)
# WHERE n.name = 'John'
# RETURN n


# MATCH (superclass:EMMOMatter)
# WHERE superclass.name = 'Device'
#     return superclass

# Get the data from file

In [ ]:

# query = f"MATCH (m:{node1} {{name: '{domain_cls}'}}), (ma:{node2} {{name: '{range_cls}'}}) CREATE (m)-[:{rel_type}]->(ma)"
#             result = session.run(query)

In [3]:
import pandas as pd
path='C://Abhishek Thesis//OneDrive_1_5-11-2023//test file-organization data//org_data_2.xlsx'
# Read Excel file and specify data types for columns
df = pd.read_excel(path)

data = []
for index, row in df.iterrows():
    attribute = row[0]
    value = row[1]
    data.append((attribute.strip(), str(value).strip()))

# print(data)


# Generate variable names dynamically
var_names = [x[0].lower().replace(' ', '_').replace('-', '_') for x in data]

# Create a string with the assignments
assign_str = '\n'.join([f"{var_names[i]} = '{data[i][1]}'" for i in range(len(data))])

# Execute the string to perform the assignments
exec(assign_str)


# Load file data to the neo4j ontology and make relationships

In [4]:
# def create_or_get_node(tx, label, **properties):
#     """ check for existing nodes, if not there, create new"""
#     query = "MERGE (n:%s {name: $name}) RETURN n" % label
#     result = tx.run(query, name=properties['name'])
#     if not result.peek():
#         create_node(tx, label, **properties)
     
    
    
def create_nodes(tx, label, **properties):
    query = f"CREATE (n:{label}) SET n += $properties RETURN n"
    return tx.run(query, properties=properties)

def create_or_get_node(tx, label, match_property, properties):
    query = f"MERGE (n:{label} {{{match_property}: $value}}) SET n += $properties RETURN n"
    result = tx.run(query, value=properties[match_property], properties=properties)
    if not result.peek():
        create_nodes(tx, label, **properties)


# Function to create the relationship if it doesn't exist
def create_relation(tx, node1, node2, relation, property1, property2):
    query = f"MATCH (n1), (n2) WHERE n1.{property1} = $name1 AND n2.{property2} = $name2 " \
            f"MERGE (n1)-[r:{relation}]->(n2) " \
            f"RETURN count(r)"
    result = tx.run(query, name1=node1, name2=node2)
    count = result.single()[0]
    return count
    
founding_body_list = founding_body.split(",")    
country_list = country.split(",")
institution_list = institution.split(",")
orcid_list = orcid.split(",")
authors_list_list = authors_list.split(",")

# Create nodes for Researcher, Institution and Country
with driver.session() as session:
    with session.begin_transaction() as tx:
    
        for country_ in country_list:
            if country not in ('','-','nan'):
                properties = {'name': country_}
                create_or_get_node(tx, "Country",'name', properties)
        
    
        for institution_ in institution_list:
            if institution not in ('','-','nan'):
                properties = {'name': institution_}
                create_or_get_node(tx, "Institution",'name', properties)

        for authors_list_,orcid_ in zip(authors_list_list,orcid_list):
            if orcid not in ('', '-', 'nan'):
                properties = {'name': authors_list_, 'orcid': orcid_}
                create_or_get_node(tx, "Researcher", 'orcid', properties)

        
#         if orcid not in ('','-','nan'): 
#             create_or_get_node(tx, "Researcher", name=authors_list, orcid=orcid)

                
        
        if subcomponent not in ('','-','nan'):    
            properties = {'name': subcomponent}
            create_or_get_node(tx, "Subcomponent",'name', properties)

                
        if component not in ('','-','nan'):    
            properties = {'name': component}
            create_or_get_node(tx, "Component",'name', properties)
  
        if device not in ('','-','nan'): 
            properties = {'name': device}
            create_or_get_node(tx, "Device",'name', properties)

        

        if material not in ('','-','nan'):  
            properties = {'name': material}
            create_or_get_node(tx, "Material", 'name',properties)

                     

        properties = {'name': measurement_id,'experiment_start':measurement_date,
                    'date_added':upload_date, 'run_title' :experiment_title, 'experiment_id':experiment_id}
        create_or_get_node(tx, "Measurement",'name',properties)

        
        
# config_rel_lst=[{'node1':orcid, 'node2':institution,'relation': 'AFFILIATED_TO', 'property_name1':'orcid','property_name2': 'name'},
#       {'node1':institution, 'node2':country,'relation': 'IN', 'property_name1':'name','property_name2': 'name'},
#       {'node1':orcid, 'node2':country,'relation': 'IN', 'property_name1':'orcid','property_name2': 'name'},
#       {'node1':measurement_id, 'node2':orcid,'relation': 'BY', 'property_name1':'name','property_name2': 'orcid'} 
#      ]
           
# Create relations between the nodes  
    with session.begin_transaction() as tx:  
    # Check and create relationship
        for orcid_,institution_  in zip(orcid_list,institution_list):
            count = create_relation(tx, orcid_, institution_, 'AFFILIATED_TO', 'orcid', 'name')

            if count == 0:
                print("Relationship AFFILIATED_TO doesn't exist, creating it...")
                create_relation(tx, orcid_, institution_, 'AFFILIATED_TO', 'orcid', 'name')
            else:
                pass

        for institution_, country_ in zip(institution_list,country_list):

            count = create_relation(tx, institution_, country_, 'IN', 'name', 'name')

            if count == 0:
                print("Relationship IN doesn't exist, creating it...")
                create_relation(tx, institution_, country_, 'IN', 'name', 'name')
            else:
                pass
        
        
        for orcid_, country_ in zip(orcid_list,country_list):
            count = create_relation(tx, orcid_, country_, 'IN', 'orcid', 'name')

            if count == 0:
                print("Relationship IN_ orcid doesn't exist, creating it...")
                create_relation(tx, orcid_, country_, 'IN', 'orcid', 'name')
            else:
                pass
        
        
        for orcid_ in orcid_list:
            count = create_relation(tx, measurement_id, orcid_, 'BY', 'name', 'orcid')

            if count == 0:
                print("Relationship BY doesn't exist, creating it...")
                create_relation(tx, measurement_id, orcid_, 'BY', 'name', 'orcid')
            else:
                pass
        



# To get the classes and subclasses from ontology, so as to create further IS_A relations with file data

In [17]:
from neo4j import GraphDatabase


def get_all_subclasses(driver, target_class_name,node_lab):
    with driver.session() as session:
#        result = session.run(
#            "MATCH (target {name: $targetClassName})<-[:$relationshipType*]-(subclass: {name: $targetClassName}) RETURN DISTINCT subclass.name",
#                    targetClassName=target_class_name,
#                    relationshipType=relationship_type)
        
        result = session.run(
            "MATCH (target:{node_lab} {{name: $targetClassName}})<-[:EMMO_IS_A*]-(subclass:{node_lab}) "
            "RETURN DISTINCT subclass.name".format(node_lab=node_lab),
            targetClassName=target_class_name
        )
        return set([record["subclass.name"] for record in result])




In [6]:
# Example usage
target_class_name = "Component"
node_lab='EMMOMatter'
# Create a Neo4j driver instance
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

# Call the function to retrieve all subclasses
all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)

# Include the target class itself in the set of subclasses
all_subclasses.add(target_class_name)

# Convert the set to a list
component_lst = list(all_subclasses)

# Print the subclasses
print(component_lst)

['AquivionSO3H-Form', 'PolarizableElectrode', 'PartiallyFluorinatedMembrane', 'AquivionD79-25BS', 'NonFluorinatedCompositeMembrane', 'NRE212', 'NegativeElectrode', 'Component', 'PlatinumElectrode', 'PorousElectrode', 'InertElectrode', 'CompositeElectrode', 'CarbonPasterElectrode', 'CounterElectrode', 'SimpleElectrode', 'Flemion', 'WorkingElectrode', 'Cathode', 'Neosepta-F', 'HydrogenElectrode', '5LayerMEA', 'NonFluorinatedMembrane', 'PositiveElectrode', 'Aquivion', 'NRE211', 'GasDiffusionElectrode', 'Asiplex', 'MetalElectrode', 'AquivionD98-25BS', 'PerfluorinatedMembrane', 'RotationDiskElectrode', 'Anode', 'AquivionD72-25BS', 'BimetallicElectrode', 'ConversionElectrode', 'NormalHydrogenElectrode', 'MembraneElectrodeAssembly', 'NafionMembrane', 'LithiumIntercalationElectrode', 'IndicatorElectrode', 'CatalystLayer', 'Membrane', 'NonPolarizableElectrode', 'GoldElectrode', 'Electrode', 'CatalystCoatedMembrane', 'Gore-Select', 'ReferenceElectrode', 'IntercalationElectrode', 'FuelCellCompone

In [7]:
target_class_name = "Device"
node_lab='EMMOMatter'
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
device_lst = list(all_subclasses)
print(component_lst)

['AquivionSO3H-Form', 'PolarizableElectrode', 'PartiallyFluorinatedMembrane', 'AquivionD79-25BS', 'NonFluorinatedCompositeMembrane', 'NRE212', 'NegativeElectrode', 'Component', 'PlatinumElectrode', 'PorousElectrode', 'InertElectrode', 'CompositeElectrode', 'CarbonPasterElectrode', 'CounterElectrode', 'SimpleElectrode', 'Flemion', 'WorkingElectrode', 'Cathode', 'Neosepta-F', 'HydrogenElectrode', '5LayerMEA', 'NonFluorinatedMembrane', 'PositiveElectrode', 'Aquivion', 'NRE211', 'GasDiffusionElectrode', 'Asiplex', 'MetalElectrode', 'AquivionD98-25BS', 'PerfluorinatedMembrane', 'RotationDiskElectrode', 'Anode', 'AquivionD72-25BS', 'BimetallicElectrode', 'ConversionElectrode', 'NormalHydrogenElectrode', 'MembraneElectrodeAssembly', 'NafionMembrane', 'LithiumIntercalationElectrode', 'IndicatorElectrode', 'CatalystLayer', 'Membrane', 'NonPolarizableElectrode', 'GoldElectrode', 'Electrode', 'CatalystCoatedMembrane', 'Gore-Select', 'ReferenceElectrode', 'IntercalationElectrode', 'FuelCellCompone

In [8]:
target_class_name = "Subcomponent"
node_lab='EMMOMatter'
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
subcomponent_lst = list(all_subclasses)
print(subcomponent_lst)

['CoatedMetallicBipolarPlate', 'AluminiumBasedGraphiteCoatedBipolarPlate', 'NickelBasedMetalNitrideCoatedBipolarPlate', 'ElectronicSubcomponent', 'Station', 'NickelBasedGraphiteCoatedBipolarPlate', 'ElectrochemicalSubcomponent', 'StainlessSteelBasedBiploarPlate', 'AluminiumBasedCarbonBasedCoatedBipolarPlate', 'SwagelokCellHousing', 'AluminiumBasedOrganicSelfAssemblyMonoPolymerCoatedBipolarPlate', 'TitaniumBasedMetalBasedCoatedBipolarPlate', 'ElectricCurrentCollector', 'AluminiumBasedMetalNitrideCoatedBipolarPlate', 'CylindricalCellHousing', 'NonPorousGraphiteBipolarPlate', 'CompositeBipolarPlate', 'NickelBasedBipolarPlate', 'AluminiumBasedDiamondLikeCarbonCoatedBipolatPlate', 'AluminiumBasedNobelMetalCoatedBipolarPlate', 'TitaniumBasedMetalCarbideCoatedBipolarPlate', 'StructuralSubcomponent', 'TitaniumBasedGraohiteCoatedBipolarPlate', 'GasDiffusionLayer', 'StainlessSteelBasedCarbonBasedCoatedBipolarPlate', 'FeltGDL', 'TitaniumBasedConductivePolymerCoatedBipolarPlate', 'TitaniumBasedMte

In [9]:
target_class_name = "Material"
node_lab='EMMOMatter'
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
all_subclasses = get_all_subclasses(driver, target_class_name,node_lab)
all_subclasses.add(target_class_name)
material_lst = list(all_subclasses)
print(material_lst)

['Metal', 'IonicLiquidElectrolyte', 'SingleMetalCatalyst', 'IsotropicMaterial', 'EA30E', 'ElectroCatalyst', 'Aquivion', 'FunctionalMaterial', 'CarbonBlack', 'Polymer', 'SecundaryCatalyst', 'CompositeMaterial', 'Catalyst', '1DNanoMaterial', 'Electrolyte', 'SupportingElectrolyte', 'F30E', 'FCarbonSupportedPlatinumCatalyst', '800EWIonomer', 'Nafion_D2020CS', 'ThermalBlack', 'AcidicElectrolyte', 'Nafion', 'LampBlack', 'AquivionD79-25BS', 'Ionomer', 'CoatingSubstrate', 'ElectrochemicalMaterial', 'ElectrochemicalIntercalationMaterial', 'HomoGenousMaterial', 'NonAqueousElectrolyte', 'LiquidElectrolyte', 'Nafion_D520CS', 'AnisotropicMaterial', 'PlatinumCatalyst', 'PTFE', 'Polyepoxide', 'AquivionSO3H-Form', 'IntercalationMaterial', 'NanoMaterial', 'FurnaceBlack', 'NaturalMaterial', 'BinaryCarbonSupportedPlatinumCatalyst', 'ElectroChemicalIntercalationMaterial', 'SolidElectrolyte', 'Material', 'F50E', 'CarbonSupportedPlatinumCatalyst', 'ChannelBlack', 'GasSolution', 'Ceramic', '3M', 'AlkalineEle

In [10]:
from neo4j import GraphDatabase

def create_subclass(driver, node_label, superclass_name, subclass_name, url):
    with driver.session() as session:
        result = session.run(
            "MATCH (superclass:" + node_label + ") "
            "WHERE superclass.name = $superclassName "
            "MERGE (subclass:EMMOMatter:SubclassOfDevice { name: $subclassName }) "
            "ON CREATE SET subclass.url = $url "
            "MERGE (subclass)-[:EMMO_IS_A]->(superclass) "
            "RETURN subclass, superclass",
            superclassName=superclass_name,
            subclassName=subclass_name,
            url=url + subclass_name  # Append subclass_name to the URL
        )
        return result.single()




# Creation of IS_A relationship of meta data (organizational data) to EMMO classes.

In [11]:
from sentence_transformers import SentenceTransformer, util
"""
This is a simple application for sentence embeddings: semantic search

We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 3 most similar sentences in the corpus.
"""

def modify_strings(string):
    modified_lst=''
   

    if len(string.split())==1:
        print('string is ',string)
        if string.isupper()==True:
            
            
            
            
            modified_lst=string
        elif string[0].isupper()==True and string[1].isupper()==False:
            modified_lst=string
#                 print(modified_string)
        else:
            modified_lst=string.title()

    else:     

        modified_string = ''.join(word.capitalize() for word in string.split())
        modified_lst=modified_string
    return modified_lst



def emmo_cls_onto(queries,emmo_lst,node_label,superclass_name,url):
    
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    # Corpus with example sentences
    cls_lst_onto = emmo_lst
    cls_str_onto = []
    for n in cls_lst_onto:
        cls_str_onto.append(str(n))
    #print(abc_str)


    corpus = cls_str_onto
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    cls_dict ={}
    # Query sentences:
    # queries = class_list
    # Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
    top_k = min(3, len(corpus))

    subclass_name = queries
#         words = subclass_name.split()
#         capitalized_words = [word.capitalize() for word in words]
#         query = ''.join(capitalized_words)
    query = modify_strings(subclass_name)
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    c_lst=[]
    scr_lst=[]

    # We use cosine-similarity and torch.topk to find the highest 3 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
#     print(top_results)
    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 3 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print('c_lst ',corpus[idx], "(Score: {:.4f})".format(score))
        c_lst.append(corpus[idx].replace('mason.', ''))
        scr_lst.append("{:.4f}".format(score)) 
#         print('c_lst ',c_lst)


    with driver.session() as session:
        with session.begin_transaction() as tx:

            if float(scr_lst[0])>=0.6:
                spr_cls = c_lst[0]  

                create_relation(tx,subclass_name ,spr_cls, "IS_A", 'name', 'name')


                print(query,c_lst[0],superclass_name)

#                 with session.begin_transaction() as tx:   
#                     if device not in ('','-','nan'):  
#                         properties = {'name': material}
#                         create_or_get_node(tx, "Material", properties)


            elif float(scr_lst[0])<0.6:     
                # Example usage

                # Create a Neo4j driver instance
                # Call the function to create the subclass and "IS_A" relationship
                create_subclass(driver, node_label, superclass_name, query, url)
                create_relation(tx, subclass_name ,query, "IS_A",  'name', 'name')

                print('outside func ',superclass_name ,node_label, query, url)
#            createNewClass(query,spr_cls)



def create_relation(tx, node1, node2, relation, property1, property2):
    query = f"MATCH (n1), (n2) WHERE n1.{property1} = $name1 AND n2.{property2} = $name2 " \
            f"MERGE (n1)-[r:{relation}]->(n2) " \
            f"RETURN count(r)"
    result = tx.run(query, name1=node1, name2=node2)
    count = result.single()[0]
    return count

In [12]:
node_label = 'EMMOMatter'
superclass_name='Device'
url = 'http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#'
# d_lst=['Catalysis','Lithium Battery','PEMFC','Fuel Cells','FUELCELLS','FuelCells','Vanadium redox flow battery','VanadiumRedoxFlowBattery','Proton Exchange Membrane Fuel Cell','LithiumBattery','Polymer electrolyte fuel cell','PolymerElectrolyteFuelCell']
cls_dict=emmo_cls_onto(device,device_lst,node_label,superclass_name,url)






Query: FuelCells

Top 3 most similar sentences in corpus:
c_lst  FuelCell (Score: 0.9472)
c_lst  FuelCellStack (Score: 0.8775)
c_lst  BatteryCell (Score: 0.6030)
FuelCells FuelCell Device


In [ ]:
# with driver.session() as session:
#     with session.begin_transaction() as tx:
#         node_label = 'EMMOMatter'
#         superclass_name='Device'
#         spr_cls='FuelCell' 
#         query='FuelCells'
#         create_relation(tx,query ,spr_cls, "IS_A", 'name', 'name')

In [13]:
node_label = 'EMMOMatter'
superclass_name='Component'
url = 'http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#'
# c_lst_=['Catalyst layer','Membrane electrode assembly','MEA','Battery Electrode']
cls_dict=emmo_cls_onto(component,component_lst,node_label,superclass_name,url)


string is  MEA




Query: MEA

Top 3 most similar sentences in corpus:
c_lst  5LayerMEA (Score: 0.3055)
c_lst  Component (Score: 0.2750)
c_lst  Aquivion (Score: 0.2700)
outside func  Component EMMOMatter MEA http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#


In [14]:
node_label = 'EMMOMatter'
superclass_name='Subcomponent'
url = 'http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#'
# s_lst_=['Catalyst','Gas diffusion electrode','Catalyst Layer']
cls_dict=emmo_cls_onto(subcomponent,subcomponent_lst,node_label,superclass_name,url)







Query: CatalystLayer

Top 3 most similar sentences in corpus:
c_lst  GasDiffusionLayer (Score: 0.4745)
c_lst  TitaniumBasedDiamondLikeCarbonCoatedBipolarPlate (Score: 0.4340)
c_lst  TitaniumBasedGraohiteCoatedBipolarPlate (Score: 0.4150)
outside func  Subcomponent EMMOMatter CatalystLayer http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#


In [15]:
node_label = 'EMMOMatter'
superclass_name='Material'
url = 'http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#'
# lst_=['30 wt% Pd/C','5 wt% Pd/C','Gas diffusion layer','0.2 mg/cm2 Pt MEA','platinum','Carbon felt electrode','NMC111 electrode','LixMnO2','Pt']
cls_dict=emmo_cls_onto(material,material_lst,node_label,superclass_name,url)





Query: 0.2Mg/cm2PtMea

Top 3 most similar sentences in corpus:
c_lst  3M (Score: 0.4049)
c_lst  LiquidElectrolyte (Score: 0.3925)
c_lst  NonAqueousElectrolyte (Score: 0.3846)
outside func  Material EMMOMatter 0.2Mg/cm2PtMea http://www.semanticweb.org/maxdreger/ontologies/2022/10/matter/#


In [ ]:
# print(Measurement_ID,Experiment_ID=Experiment_ID, Experiment_Title=Experiment_Title, Measurement_date=Measurement_date)